In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

# 📁 Folder Setup
input_folder = r'C:\Users\jahna\OneDrive\Documents\TIME_SERIES_STOCK_FORECASTING\PREPROCESSING\CLEAN_DATA_FINAL'  
output_folder = r'C:\Users\jahna\OneDrive\Documents\TIME_SERIES_STOCK_FORECASTING\Time_Series_Models\time_series_outputs'
model_folder = os.path.join(output_folder, "models")
os.makedirs(output_folder, exist_ok=True)
os.makedirs(model_folder, exist_ok=True)

# 📊 Evaluation Function
def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return mae, rmse

# 📦 Store Metrics
all_metrics = []

# 🔁 Loop through files
for file in os.listdir(input_folder):
    if not file.endswith(".csv"):
        continue

    stock = file.replace(".csv", "")
    filepath = os.path.join(input_folder, file)

    try:
        df = pd.read_csv(filepath)
        if 'date' not in df.columns or 'close' not in df.columns:
            print(f"❌ Skipping {stock}: missing columns.")
            continue

        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df[['date', 'close']].dropna().sort_values('date')
        df.set_index('date', inplace=True)
        ts = df.asfreq('D').interpolate()

        train_size = int(len(ts) * 0.8)
        train, test = ts.iloc[:train_size], ts.iloc[train_size:]

        # 1️⃣ ARIMA
        try:
            arima_model = ARIMA(train, order=(5, 1, 0)).fit()
            arima_pred = arima_model.forecast(steps=len(test))
            arima_mae, arima_rmse = evaluate(test['close'], arima_pred)
            joblib.dump(arima_model, os.path.join(model_folder, f"{stock}_ARIMA.pkl"))
        except Exception as e:
            arima_mae, arima_rmse = None, None
            print(f"ARIMA failed for {stock}: {e}")

        # 2️⃣ SARIMA
        try:
            sarima_model = SARIMAX(train, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12)).fit(disp=False)
            sarima_pred = sarima_model.forecast(steps=len(test))
            sarima_mae, sarima_rmse = evaluate(test['close'], sarima_pred)
            joblib.dump(sarima_model, os.path.join(model_folder, f"{stock}_SARIMA.pkl"))
        except Exception as e:
            sarima_mae, sarima_rmse = None, None
            print(f"SARIMA failed for {stock}: {e}")

        # 3️⃣ Prophet
        try:
            prophet_df = train.reset_index().rename(columns={"date": "ds", "close": "y"})
            test_df = test.reset_index().rename(columns={"date": "ds", "close": "y"})
            model_prophet = Prophet(daily_seasonality=True)
            model_prophet.fit(prophet_df)
            future = model_prophet.make_future_dataframe(periods=len(test))
            forecast = model_prophet.predict(future)
            prophet_pred = forecast['yhat'][-len(test):].values
            prophet_mae, prophet_rmse = evaluate(test_df['y'].values, prophet_pred)
            joblib.dump(model_prophet, os.path.join(model_folder, f"{stock}_Prophet.pkl"))
        except Exception as e:
            prophet_mae, prophet_rmse = None, None
            print(f"Prophet failed for {stock}: {e}")

        # 📈 Plot Forecasts
        plt.figure(figsize=(12, 5))
        plt.plot(test.index, test['close'], label="Actual")
        if arima_mae is not None: plt.plot(test.index, arima_pred, label="ARIMA")
        if sarima_mae is not None: plt.plot(test.index, sarima_pred, label="SARIMA")
        if prophet_mae is not None: plt.plot(test.index, prophet_pred, label="Prophet")
        plt.title(f"{stock} - Forecast Comparison")
        plt.legend()
        plt.tight_layout()
        plt.grid(True)
        plt.savefig(os.path.join(output_folder, f"{stock}_forecast.png"))
        plt.close()

        # 📊 Save metrics
        all_metrics.append({
            "Stock": stock,
            "ARIMA_MAE": arima_mae,
            "ARIMA_RMSE": arima_rmse,
            "SARIMA_MAE": sarima_mae,
            "SARIMA_RMSE": sarima_rmse,
            "Prophet_MAE": prophet_mae,
            "Prophet_RMSE": prophet_rmse,
        })

        print(f"✅ {stock} processed.")

    except Exception as e:
        print(f"⚠️ Skipping {stock} due to error: {e}")

# 💾 Save metrics
metrics_df = pd.DataFrame(all_metrics)
metrics_df.to_csv(os.path.join(output_folder, "model_metrics_summary.csv"), index=False)


print("🎉 All models processed, saved, and zipped.")


15:48:49 - cmdstanpy - INFO - Chain [1] start processing
15:48:51 - cmdstanpy - INFO - Chain [1] done processing


✅ ADANIPORTS processed.


15:49:22 - cmdstanpy - INFO - Chain [1] start processing
15:49:26 - cmdstanpy - INFO - Chain [1] done processing


✅ ASIANPAINT processed.



KeyboardInterrupt

